## Goal: What is Fst score per exon + ss (+-2bp)
For exons that are 
* unique to one population or the other
* shared between the population

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot


p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *

In [4]:
config = load_config()
od = '../'

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

In [6]:
df = pd.read_csv(proc_cfg(config['lr']['exons_ss_in_pops_fsts_summary'], od),
                 sep='\t')
df.rename({'ThickStart':'eid',
           'ThickEnd':'novelty',
           'ItemRGB':'pop1_det',
           'BlockCount':'pop2_det'}, 
          axis=1, inplace=True)

,Chromosome,Start,End,Name,Score,Strand,eid,novelty,pop1_det,pop2_det,End_b,fst,Start_b,pop1,pop2
0,GL000008.2,135133,135173,.,.,+,GL000008.2_+_135133_135173,Known,True,False,-1,-1.0,-1,PEL,YRI
1,GL000008.2,155429,155531,.,.,+,GL000008.2_+_155429_155531,Known,True,False,-1,-1.0,-1,PEL,YRI
2,GL000008.2,83926,84014,.,.,+,GL000008.2_+_83926_84014,Known,True,False,-1,-1.0,-1,PEL,YRI
3,GL000008.2,83926,84145,.,.,+,GL000008.2_+_83926_84145,Known,True,True,-1,-1.0,-1,PEL,YRI
4,GL000008.2,85442,85477,.,.,+,GL000008.2_+_85442_85477,Known,True,False,-1,-1.0,-1,PEL,YRI


In [12]:
# set all <0 fsts --> 0 https://www.biostars.org/p/132253/
df.loc[df.fst<0, 'fst'] = 0

In [16]:
# add up each fst per contrast and exon
df = df[['eid', 'novelty', 'pop1_det', 'pop2_det',
         'fst', 'pop1', 'pop2']]

,Chromosome,Start,End,Name,Score,Strand,eid,novelty,pop1_det,pop2_det,End_b,fst,Start_b,pop1,pop2,elen
0,GL000008.2,135133,135173,.,.,+,GL000008.2_+_135133_135173,Known,True,False,-1,0.0,-1,PEL,YRI,40
1,GL000008.2,155429,155531,.,.,+,GL000008.2_+_155429_155531,Known,True,False,-1,0.0,-1,PEL,YRI,102
2,GL000008.2,83926,84014,.,.,+,GL000008.2_+_83926_84014,Known,True,False,-1,0.0,-1,PEL,YRI,88
3,GL000008.2,83926,84145,.,.,+,GL000008.2_+_83926_84145,Known,True,True,-1,0.0,-1,PEL,YRI,219
4,GL000008.2,85442,85477,.,.,+,GL000008.2_+_85442_85477,Known,True,False,-1,0.0,-1,PEL,YRI,35


In [ ]:
# get length of each exon
df['elen'] = df['End']-df['Start']